In [2]:
# Importing packages

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Reading data

df = pd.read_excel(r'C:\Users\anant\OneDrive\Desktop\MBAN\MMAI 5000\sfd_data.xlsx')

In [4]:
df.columns

Index(['X', 'Y', 'OBJECTID', 'EVENT_UNIQUE_ID', 'OCC_DATE', 'OCC_YEAR',
       'OCC_MONTH', 'OCC_DOW', 'OCC_DOY', 'OCC_DAY', 'OCC_HOUR',
       'OCC_TIME_RANGE', 'DIVISION', 'DEATH', 'INJURIES', 'HOOD_158',
       'NEIGHBOURHOOD_158', 'HOOD_140', 'NEIGHBOURHOOD_140', 'LONG_WGS84',
       'LAT_WGS84'],
      dtype='object')

In [5]:
# Check for nulls

df.isnull().any()

X                    False
Y                    False
OBJECTID             False
EVENT_UNIQUE_ID      False
OCC_DATE             False
OCC_YEAR             False
OCC_MONTH            False
OCC_DOW              False
OCC_DOY              False
OCC_DAY              False
OCC_HOUR             False
OCC_TIME_RANGE       False
DIVISION             False
DEATH                False
INJURIES             False
HOOD_158             False
NEIGHBOURHOOD_158    False
HOOD_140             False
NEIGHBOURHOOD_140    False
LONG_WGS84           False
LAT_WGS84            False
dtype: bool

In [7]:
# Identifying trends in Deaths and Injuries

df.groupby(['OCC_YEAR']).agg(
    {'DEATH' : 'sum',
    'INJURIES' : 'sum',
    'OBJECTID' : 'count'}
).reset_index().rename({'OBJECTID' : 'Incidents'}, axis = 1)

,OCC_YEAR,DEATH,INJURIES,Incidents
0,2004,26,96,191
1,2005,53,178,262
2,2006,29,173,215
3,2007,44,129,207
4,2008,37,171,238
5,2009,37,144,252
6,2010,32,138,259
7,2011,28,113,227
8,2012,34,114,219
9,2013,22,119,204


In [11]:
# Removing redundant columns

df.drop(columns = ['X', 'Y', \
   'EVENT_UNIQUE_ID', \
     'OCC_DATE', \
        'HOOD_140', 'NEIGHBOURHOOD_140'], axis = 1, inplace = True)

In [13]:
# Removing missing data

df_filtered = df[df['NEIGHBOURHOOD_158'] != 'NSA']

In [19]:
# Creating new target variable

df_filtered['Victims'] = df_filtered['DEATH'] +  df_filtered['INJURIES']

C:\Users\anant\AppData\Local\Temp\ipykernel_46492\1784834653.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Victims'] = df_filtered['DEATH'] +  df_filtered['INJURIES']


In [21]:
# Removing outliers

df_filtered.sort_values(by = ['Victims'], ascending = False)

,OBJECTID,OCC_YEAR,OCC_MONTH,OCC_DOW,OCC_DOY,OCC_DAY,OCC_HOUR,OCC_TIME_RANGE,DIVISION,DEATH,INJURIES,HOOD_158,NEIGHBOURHOOD_158,LONG_WGS84,LAT_WGS84,Victims
1853,1854,2012,July,Monday,198,16,0,Night,D43,2,24,136,West Hill,-79.189606,43.767647,26
3727,3728,2018,July,Monday,204,23,0,Night,D54,2,13,67,Playter Estates-Danforth,-79.351474,43.678046,15
451,452,2005,December,Monday,360,26,0,Night,D52,1,6,170,Yonge-Bay Corridor,-79.380755,43.652120,7
1346,1347,2009,May,Sunday,130,10,3,Night,D22,0,6,13,Etobicoke West Mall,-79.569720,43.651586,6
5356,5357,2022,June,Tuesday,179,28,1,Night,D41,0,6,126,Dorset Park,-79.274649,43.776939,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2884,2885,2016,December,Tuesday,355,20,10,Morning,D54,0,0,61,Taylor-Massey,-79.295132,43.696074,0
1,2,2004,July,Monday,208,26,23,Evening,D23,0,0,1,West Humber-Clairville,-79.618218,43.733547,0
2887,2888,2016,January,Friday,1,1,6,Morning,D33,0,0,46,Pleasant View,-79.332505,43.778844,0
2889,2890,2016,May,Thursday,126,5,0,Night,D55,0,0,70,South Riverdale,-79.352919,43.660834,0


In [24]:
# Function to remove outliers

def remove_outliers(df, column):

    # Calculate quartile 1 and quartile 3
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)

    # Calculate interquartile range
    IQR = q3 - q1

    # Define lower and upper bounds
    lower_bound = q1 - 1.5 * IQR
    upper_bound = q3 + 1.5 * IQR

    # Filter outliers
    filtered_df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

    return filtered_df

In [25]:
# Final dataset

ads = remove_outliers(df_filtered, 'Victims')